In [30]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-12-09 13:46:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-12-09 13:46:51 (19.3 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [31]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

In [32]:
print("The length of the dataset is: ", len(text))

The length of the dataset is:  1115394


In [33]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [34]:
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [35]:
stoi = {character:integer for integer, character in enumerate(characters)}
itos = {integer:character for integer, character in enumerate(characters)}

encode = lambda string: [stoi[character] for character in string]  # Encoder -> convert a string to a list of integers
decode = lambda list: ''.join([itos[integer] for integer in list]) # Decoder -> take a list of integers and convert it into a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [36]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x1191d1450>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15,

In [37]:
n = int(0.9 * len(data))
training_data = data[:n]
validation_data = data[n:]

In [38]:
block_size = 8 # Context length
training_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [39]:
x = training_data[:block_size]
y = training_data[1:block_size + 1]
for token in range(block_size):
    context = x[:token + 1]
    target = y[token]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [40]:
torch.manual_seed(1337)
batch_size = 4 # the number of sequences we process in parallel
block_size = 8 # the maixmum context length for each prediction

def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [[24, 43, 58, 5, 57, 1, 46, 43]] the target: 43
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58]] the target: 58
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1]] the target: 5
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [25, 17, 27, 10, 0, 21, 1, 54]] the target: 57
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [25, 17, 27, 10, 0, 21, 1, 54]] the target: 1
wh

In [41]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = 1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [42]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVjX-YBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!murI!IM?SPNPq?VgC'R
pD3cLv-bxn-tL!upg
SZ!Uvdg CtxtT?hsiW:XxKIiPlagHIsr'zKSVxza?GlDWObPmRJgrIAcmspmZ&viCKot:u3qYXA:rZgv f:3Q-oiwUzqh'Z!I'zRS3SP rVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i
YxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph
LBj,KZxJaLPgBAkyzEzSiiQb
jkSVyb$vvyQFuAUAKuzdZAJktRqUiAcPBa;AgJ;.$l3Pu!.IErMfN!PmuQbvx
xMkttN:PmJh'wNC
AUI?wNCphq-.IsCwbjxca;P-KA:r'a;pJ&q-UgOEX.cAO-p,lQ?nEsrlvmUgbEQLQh,j;iPlgZR:CJpxIBju f&!BBEHSPmnq,P -d
pjuWDPLFa!ByCSjJuERtKpph.ZP  CUEsiy'FjF$$-rJUQ?uApxlxlYe
yASBoipGLwfXelgY!a fyFPJX!JDWCoAXRJJFJOlxlvpR?OXYddZAXzkIBtp3d,vAcPlgX'pM fNMLphx&flaAcL!3F.?sBiRwLTqHzot.ttRF$Fv'bL:&x&ayFVqVAHqHxv3QzteqbcUJnERZYGwzLd,rgf&yCnERErI.IVZ WddEJAX CO'Eu!I!Lg:i-$ mIc.
xJjdLEJXVTb?Eqf IgCJcUGNSBZ3dbsEXgCPmr'XxxDXXxE

In [43]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [44]:
batch_size = 32
for steps in range(100):
    # Get a sample batch of data
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    print(loss.item())

4.715392112731934
4.781221389770508
4.645686149597168
4.7960920333862305
4.783337116241455
4.711153507232666
4.701266765594482
4.612026691436768
4.838379859924316
4.734117031097412
4.707516193389893
4.7707905769348145
4.804861068725586
4.559694290161133
4.701857089996338
4.742461681365967
4.672660827636719
4.653083324432373
4.7315592765808105
4.763871669769287
4.6863532066345215
4.676277160644531
4.7206339836120605
4.719085216522217
4.681148529052734
4.7270073890686035
4.7336225509643555
4.659067153930664
4.689385414123535
4.747793197631836
4.6440205574035645
4.707948207855225
4.70572566986084
4.674625396728516
4.677109718322754
4.644956111907959
4.6193084716796875
4.671792030334473
4.713363170623779
4.76865291595459
4.648808479309082
4.686805725097656
4.621817111968994
4.650778770446777
4.643256664276123
4.701192378997803
4.743958950042725
4.640769958496094
4.633058071136475
4.693822383880615
4.665843963623047
4.7592267990112305
4.701364994049072
4.645552158355713
4.69441032409668
4.7

In [45]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


juCoTxc3fAatlEKhV&fqo$FEN
mUC fq.LAXY!&ycIDDWT:lviqmVI&ydE?qfgFd,R:C&aIBUjwIFhrLIn,l,SmZzmy'zY,WuEr-tpJG-gYQThXtN3XNAc;KMbObHRlylTyuF-d,';LGjF&sr'Rjj?jK$?Y!$dD.i
SaQJpOlytJ;'thtriVc'AeDnYUXeawa!uvqHWcsV.uDzHQH:CVJxrhsgUv&y
GbOJQEXotow:'uM qfT'aI-apDqIVlHnUQYsBJ$wLgnqfI!qKoi f'sBN:WKjKBylfRqPN&QTVqr&y'rLftimb
XXgCCnpM:CjRvvV&bC-FtoisyuQ&fSVwUApxEsTVEwa!uZgXXlxitoCU'PwhrIfkId?OEo.RdKo?nQisrdnuGNpx$pZGRr&yyh$-IPy'Ih -Yu$G.pJMOE!q?Z3evX-KAk.IBT!mVjnm
qMDMphYya!UNXA'gFH-bk$Mp$DV,,e:COs&vEsV3m,uybPd?pUGRlx
pxyTsStepkgf?IiHTbR
V?uj?Y3i.aUiGGviClI!U;A-QQW:sk!lj$ Plg
wIFERw fDAAJO'h;
,phsVHxBo.
3n;pJiGJK$ AiMAcf?Ko
DTAkNe?lg?axfrOlIcfsrd?ID?aNCdr:ChOsCbftyhx? pJtxBOItoiA3:CDXtKFd&yQ?wbVj
YmzoIgjuDHWTXI&lvL'aM&yDWWWQ!ulvChs fZ'q-wyXER
o,pfjujul;
FtaepmI'aw!ae:wfyxI$ajukH-tSS&ukEZBDq,f:COLQKzPA'yboqUiAkzd ipuMx'MUA!pmuoiPanqYQEszkR
cUgCjgqffaHStNup-gwa!zMga-$vHBfRZR:xq-gU?R'AcCbRj?pGGJKuCnp$dXn'm fawNzGX?-QA.siSqU FV&$-$Op.ybPUA'ME;D&'nJhp$LlgUDnev&yF$FVbvnOJm; ck$C&fqThhWKfIT:h fB.yFQ&cIM:CKWtO

In [46]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [47]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [48]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [49]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [50]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B, T, C) ---> (B, T, C)
torch.allclose(xbow, xbow2) 

True

In [51]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [58]:
# Version 4: Self-attention...
torch.manual_seed(1337)
B,T,C = 4, 8, 2
x = torch.randn(B, T, C)

# single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v

#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [59]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5599, 0.4401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3220, 0.2016, 0.4764, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1640, 0.0815, 0.2961, 0.4585, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2051, 0.3007, 0.1894, 0.1808, 0.1241, 0.0000, 0.0000, 0.0000],
        [0.0600, 0.1273, 0.0291, 0.0169, 0.0552, 0.7114, 0.0000, 0.0000],
        [0.1408, 0.1025, 0.1744, 0.2038, 0.1690, 0.0669, 0.1426, 0.0000],
        [0.0223, 0.1086, 0.0082, 0.0040, 0.0080, 0.7257, 0.0216, 0.1016]],
       grad_fn=<SelectBackward0>)